In [1]:
import tweepy
import configparser 
import pandas as pd
from tinydb import TinyDB, Query, Storage
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy
from pyvis.network import Network


In [2]:
config = configparser.ConfigParser(interpolation=None)
config.read('config.ini')

bearer_token = config['twitter']['bearer_token']
client = tweepy.Client(bearer_token=bearer_token, return_type = dict, wait_on_rate_limit=True)


### Getting a basic list of tweets

In [3]:
tweets = client.search_recent_tweets("barcelona", expansions=['author_id'], user_fields=['username'], max_results=10)

In [4]:
users_df = pd.DataFrame(tweets['includes']['users'])
data_df = pd.DataFrame(tweets['data'])
data_df = pd.merge(data_df, users_df, left_on=['author_id'], right_on=['id'])
data_df

,text,id_x,edit_history_tweet_ids,author_id,id_y,name,username
0,Manchester United 3 Barcelona 0\nECWC Q/F 2nd ...,1603790664093011969,[1603790664093011969],2758212142,2758212142,Mal Winkles - Football Nostalgia,MemorabiliaMal
1,RT @brfootball: This mural of Achraf Hakimi an...,1603790662922649606,[1603790662922649606],939819938,939819938,Taha Muhammad,tahamuhammad95
2,RT @brfootball: This mural of Achraf Hakimi an...,1603790662222360576,[1603790662222360576],3499408878,3499408878,Calon Jenazah,khoirulwarisin_
3,RT @FabrizioRomano: Amrabat: “I’m proud to be ...,1603790661949575170,[1603790661949575170],1475823556110045187,1475823556110045187,Lory Gagliotta,LG24Sports
4,"Ousmane Dembele 🗣 :""Messi olağanüstü bir oyunc...",1603790657285509120,[1603790657285509120],2450463800,2450463800,Barcelona Turkey,FCBTURK
5,@OficialSala12 🏴󠁧󠁢󠁥󠁮󠁧󠁿 Chelsea: Drogba \n🏴󠁧󠁢󠁥󠁮...,1603790656404688896,[1603790656404688896],721053855442726913,721053855442726913,th,th14goef
6,RT @ArsenalBuzzCom: 🗞️| @standardsport: “Memph...,1603790650675191808,[1603790650675191808],1173751473374670848,1173751473374670848,Neyogunnersz,neyogunnersz
7,Se me hace complicado hacer uso del transporte...,1603790650016731137,[1603790650016731137],86278057,86278057,Txus Ballesteros 🦎,txusballesteros
8,Se planta y no quiere más jugadores del repres...,1603790646757785600,[1603790646757785600],4838595123,4838595123,Mi Otra Liga,MiOtraLiga_MOL
9,"RT @LaMusikalite: ""Barcelona no estaba prepara...",1603790643561697280,[1603790643561697280],4274275815,4274275815,Esther ⚡️🪂,esthercobosm


### Analyzing users declared networks

#### Getting all followers and following relationships for the users

We first obtain a small dataframe with three users. The first from the previous dataframe, a follower, and a follower from this second user.

In [5]:
graph_users_df = pd.DataFrame()
graph_users_df = graph_users_df.append(client.get_users_followers(users_df.iloc[0]['id'])['data'][0], ignore_index = True)
graph_users_df = graph_users_df.append(client.get_users_followers(graph_users_df.iloc[0]['id'])['data'][0], ignore_index = True)
graph_users_df = graph_users_df.append(client.get_users_followers(graph_users_df.iloc[1]['id'])['data'][0], ignore_index = True)

/tmp/ipykernel_3059/599280107.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  graph_users_df = graph_users_df.append(client.get_users_followers(users_df.iloc[0]['id'])['data'][0], ignore_index = True)
/tmp/ipykernel_3059/599280107.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  graph_users_df = graph_users_df.append(client.get_users_followers(graph_users_df.iloc[0]['id'])['data'][0], ignore_index = True)
/tmp/ipykernel_3059/599280107.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  graph_users_df = graph_users_df.append(client.get_users_followers(graph_users_df.iloc[1]['id'])['data'][0], ignore_index = True)


Then we get all of these users in a simple data structure represented on a document oriented nosql database (Tinydb). The structure looks like:

"3391": {"id": "1508297808000032776", "username": "KeavieRominger", "name": "Keavie Rominger", "followers": [ " list of followers "], "followed_by": [ "list of accounts the user is following" ] }

In [6]:
!rm ./db.json
db = TinyDB('./db.json')

for index, row in graph_users_df.iterrows():

    followers_list = []
    for response in tweepy.Paginator(client.get_users_followers,
                                    id = row['id'],
                                    max_results=1000, 
                                    limit=10):
        for user in response['data']:
            followers_list.append(user)

    followed_by_list = []
    for response in tweepy.Paginator(client.get_users_following,
                                    id = row['id'],
                                    max_results=1000, 
                                    limit=10):
        for user in response['data']:
            followed_by_list.append(user)

    User = Query()

    for follower in followers_list:
        if not (db.search(User.username == follower['username'])):
            db.insert({'id': follower['id'], 'username': follower['username'], 'name': follower['name']})

    for followed in followed_by_list:
        if not (db.search(User.username == followed['username'])):
            db.insert({'id': followed['id'], 'username': followed['username'], 'name': followed['name']}) 

    if not (db.search(User.username == row['username'])):
        db.insert({'id': row['id'], 'username': row['username'], 'name': row['name'], 'followers': 
            [d['username'] for d in followers_list], 'followed_by': [d['username'] for d in followed_by_list]})
    else:
        db.update_multiple([
            ({'followers': [d['username'] for d in followers_list]}, User.username == row['username']),
            ({'followed_by': [d['username'] for d in followed_by_list]}, User.username == row['username']),
        ])

#### Graph generation

In [ ]:
G = nx.DiGraph()

User = Query()
users_w_followers = db.search(User.followers.exists())
users_w_followed_by = db.search(User.followed_by.exists())

for user in users_w_followers:
        for follower in user['followers']:
                G.add_edge(follower,  user['username'], color='green')

for user in users_w_followed_by:
        for followed_by in user['followed_by']:
                G.add_edge(user['username'], followed_by, color='blue')

nt = Network('1300px', '600px', select_menu = True, directed=True)
nt.from_nx(G)
nt.show_buttons(filter_=['physics'])
nt.show('nx.html')